Laboratorio 3
integrantes:

- Francis Aguilar - 22243
- César López - 22535
- Angela García -22869
 
enlace al repositorio: https://www.kaggle.com/code/angelargd8/lab3-ds

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import pickle
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
from PIL import Image

## Análisis exploratorio

In [ ]:
print("Contenido de /kaggle/input:")
print(os.listdir("/kaggle/input"))

In [ ]:
base_root = "/kaggle/input/mnist-multiple-dataset-comprehensive-analysis"

print("Contenido dentro del dataset:")
print(os.listdir(base_root))

In [ ]:
poly_path = "/kaggle/input/mnist-multiple-dataset-comprehensive-analysis/PolyMNIST"
print("Contenido en PolyMNIST:", os.listdir(poly_path))

In [ ]:
mmnist_path = os.path.join(poly_path, "MMNIST")
print("Contenido en MMNIST:", os.listdir(mmnist_path))

In [ ]:
train_path = os.path.join(mmnist_path, "train")
print("Contenido en train:", os.listdir(train_path))

El conjunto de datos de polyMNIST tiene cinco modalidades distintas. El fondo de cada modalidad se compone de parches aleatorios recortados de una imagen más grande, con el dígito colocado aleatoriamente dentro de estos parches. Esta configuración proporciona a cada modalidad información única de su imagen de fondo, mientras que el dígito sirve como información compartida entre todas las modalidades. Un desafío adicional, en comparación con el PolyMNIST original, es la traducción aleatoria de los dígitos.

Descripción tomada de: https://www.kaggle.com/datasets/agungpambudi/mnist-multiple-dataset-comprehensive-analysis/data

En el conjunto de datos, algo importante de ver antes de colocar los datos en un dataframe, es que el nombre de los archivos tiene el siguiente formato:

**id.etiqueta.png**


In [ ]:
#leer las imagenes del dataset y extraer sus etiquetas 
base_path = "/kaggle/input/mnist-multiple-dataset-comprehensive-analysis/PolyMNIST/MMNIST/train"

data = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".png"):
            full_path = os.path.join(root, file)
            label = file.split('.')[1]
            data.append((full_path, int(label)))

df = pd.DataFrame(data, columns=["filepath", "label"])
print(df.head(), "\nTotal imágenes:", len(df))


In [ ]:
print("Primeras filas del DataFrame:")
print(df.head())


In [ ]:
# Ver tamaño del dataset
print("\nNúmero total de imágenes:", len(df))


In [ ]:
#imagenes por cada carpeta
df['folder'] = df['filepath'].apply(lambda x: x.split('/')[-2])
sns.countplot(data=df, x="folder")
plt.title("Distribución por fuente (carpeta)")
plt.show()

In [ ]:
sns.countplot(data=df, x="label")
plt.title("Distribución de clases")
plt.xlabel("Etiqueta")
plt.ylabel("Cantidad de imágenes")
plt.show()

La distribución de clases no parece tener un gran desbalance, sin embargo es ideal que este balanceada. En este caso se usara unsersampling, ya que son muchos datos y no tienen una gran diferencia en cuanto datos.

In [ ]:
#balancear las clases, esta vez undersampling porque muy grande el dataset
from sklearn.utils import resample

min_count = df['label'].value_counts().min()
df = pd.concat([
    resample(df[df['label'] == label], replace=False, n_samples=min_count, random_state=42)
    for label in df['label'].unique()
])

In [ ]:
sns.countplot(data=df, x="label")
plt.title("Distribución de clases")
plt.xlabel("Etiqueta")
plt.ylabel("Cantidad de imágenes")
plt.show()

In [ ]:
# Duplicados por nombre
print("Duplicados:", df.duplicated("filepath").sum())

In [ ]:
# Tamaños de imagen
df['size'] = df['filepath'].apply(lambda path: Image.open(path).size)
print("Tamaños únicos:", df['size'].value_counts())

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Asegura que la columna 'label' es int
df['label'] = df['label'].astype(int)

# Obtener clases únicas
unique_labels = sorted(df['label'].unique())

# Mostrar una imagen por clase (máximo 10 si quieres limitar)
plt.figure(figsize=(12, 4))

for i, label in enumerate(unique_labels[:10]):
    subset = df[df['label'] == label]
    if not subset.empty:
        img_path = subset.iloc[0]['filepath']
        img = Image.open(img_path)
        plt.subplot(2, 5, i + 1)
        plt.imshow(img, cmap='gray')
        plt.title(f"Etiqueta: {label}")
        plt.axis('off')

plt.tight_layout()
plt.suptitle("Ejemplo visual por clase", y=1.05)
plt.show()

En los ejemplos, se logra observar que dataset tiene una alta variabilidad visual y estilo porque cada uno tiene colores, fondos, tipografía, estilo de escritura y ruido. Y hay ciertos números que puede que por su fondo que tienen mucho ruido pueda causar problemas con el entrenamiento.

#### Preparacion de datos

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

# Ruta base
base_path = "/kaggle/input/mnist-multiple-dataset-comprehensive-analysis/PolyMNIST/MMNIST"

# Altura, ancho e input shape (ajustar si cambia)
img_height, img_width = 28, 28
input_shape = (img_height, img_width, 1)  # grayscale
batch_size = 64


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

img_height, img_width = 28, 28
base_path = "/kaggle/input/mnist-multiple-dataset-comprehensive-analysis/PolyMNIST/MMNIST"
modalidades = ['m0', 'm1', 'm2', 'm3', 'm4']

def cargar_datos(modalidades, tipo='train'):
    X = []
    y = []

    for mod in modalidades:
        path = os.path.join(base_path, tipo, mod)
        archivos = sorted(os.listdir(path))

        for nombre in tqdm(archivos, desc=f'Cargando {tipo}/{mod}'):
            if nombre.endswith('.png'):
                etiqueta = int(float(nombre.split('.')[1]))
                img_path = os.path.join(path, nombre)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (img_width, img_height))  # Asegura tamaño uniforme
                img = img.astype('float32') / 255.0  # Normalizar

                X.append(img)
                y.append(etiqueta)

    X = np.expand_dims(np.array(X), -1)
    y = np.array(y)
    return X, y

# Cargar datos de entrenamiento y prueba desde modalidad m0
X_train, y_train = cargar_datos(modalidades=['m0', 'm1', 'm2', 'm3', 'm4'], tipo='train')
X_test, y_test = cargar_datos(modalidades=['m0', 'm1', 'm2', 'm3', 'm4'], tipo='test')
print("Clases:", np.unique(y_train))


In [ ]:
print("Train:", X_train.shape, y_train.shape)
print("Test:", X_test.shape, y_test.shape)

In [ ]:
# Configuración básica
altura, ancho, canales = img_height, img_width, 1
num_clases = len(np.unique(y_train))

#### Funcion para evaluar modelos

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import numpy as np

def evaluar_modelo(modelo, X_test, y_test, nombre="Modelo"):
    print(f"\n=== Evaluación de {nombre} ===")
    loss, acc = modelo.evaluate(X_test, y_test, verbose=0)
    print(f"Accuracy: {acc:.4f} - Loss: {loss:.4f}\n")

    y_pred = modelo.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        print(classification_report(y_test, y_pred_classes, zero_division=0))

    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusión - {nombre}')
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta verdadera')
    plt.show()


#### Modelo CNN - 1

In [ ]:
model1 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(altura, ancho, canales)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_clases, activation='softmax')
])

model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
evaluar_modelo(model1, X_test, y_test, "CNN 1")

#### Modelo CNN - 2

In [ ]:
model2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(altura, ancho, canales)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Dropout(0.3),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_clases, activation='softmax')
])

model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
evaluar_modelo(model2, X_test, y_test, "CNN 2")

#### Conclusion:
Luego de entrenar y comparar ambos modelos, notamos que los dos tienen un rendimiento bastante bueno al reconocer los dígitos. Sin embargo, el segundo modelo (CNN 2) fue un poco más preciso, especialmente en la validación con datos que no había visto antes. También tuvo un error menor y menos confusión entre los números parecidos. Por eso, decidimos quedarnos con el modelo CNN 2, ya que es más robusto y generaliza mejor en nuevas imágenes.



#### Modelo Red Neuronal Simple

In [ ]:
model_simple = models.Sequential([
    layers.Flatten(input_shape=(altura, ancho, canales)),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_clases, activation='softmax')
])

model_simple.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_simple.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))